In [27]:
import pandas as pd
import openai
import json
import time
from typing import Dict, List
import random
from tqdm import tqdm
import os
from datetime import datetime

class LinkedInProfileGenerator:
    def __init__(self, api_key: str):
        openai.api_key = api_key
        self.retry_delay = 5
        self.max_retries = 3
        self.output_file = "generated_profiles_NEW.csv"
        self.used_indices = set()  # Track used indices instead of locations

        self.columns = [
            'Intro', 'Full Name', 'Workplace', 'Location', 'Connections', 'Photo',
            'Followers', 'About', 'Experiences', 'Number of Experiences',
            'Educations', 'Number of Educations', 'Licenses', 'Number of Licenses',
            'Volunteering', 'Number of Volunteering', 'Skills', 'Number of Skills',
            'Recommendations', 'Number of Recommendations', 'Projects', 'Number of Projects',
            'Publications', 'Number of Publications', 'Courses', 'Number of Courses',
            'Honors', 'Number of Honors', 'Scores', 'Number of Scores',
            'Languages', 'Number of Languages', 'Organizations', 'Number of Organizations',
            'Interests', 'Number of Interests', 'Activities', 'Number of Activities',
            'Label'
        ]

    def select_training_examples(self, example_profiles: List[Dict], num_examples: int = 3) -> List[Dict]:
        # Get legitimate profiles
        legitimate_profiles = [p for p in example_profiles if p.get('Label') == 0]
        
        # Simply select 5 random profiles from legitimate ones
        selected = random.sample(legitimate_profiles, num_examples)
        
        return selected

    def create_prompts(self, example_profiles: List[Dict]) -> List[Dict]:
        examples = self.select_training_examples(example_profiles)
        return [
            {"role": "system", "content": """You are generating diverse LinkedIn profiles. Create profiles that are:
            - From different regions (North America, Europe, Asia, Africa, and Latin America etc)
            - From diverse industries (e.g., tech, healthcare, entertainment, finance, education, retail, manufacturing, arts, law, hospitality, etc.)
            - At different career stages
            - With different educational backgrounds
            - Working at different types of organizations (startups, corporations, non-profits, etc.)"""},
            {"role": "user", "content": f"""Here are some example LinkedIn profiles for format reference:
{json.dumps(examples, indent=2)}

Create a new LinkedIn profile that is similar to but different from these examples.
Use the same format but make the profile unique and realistic"""}
        ]

    def clean_field(self, field_value):
        if isinstance(field_value, str) and field_value.startswith('{') and field_value.endswith('}'):
            try:
                return json.loads(field_value.replace("'", '"'))
            except:
                return {}
        return field_value

    def generate_single_profile(self, example_profiles: List[Dict], retry_count: int = 0) -> Dict:
        try:
            response = openai.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=self.create_prompts(example_profiles),
                temperature=1.1,
                max_tokens=3000
            )

            profile_str = response.choices[0].message.content.strip()
            if profile_str.startswith("```json"):
                profile_str = profile_str.replace("```json", "").replace("```", "").strip()

            try:
                profile = json.loads(profile_str)
            except json.JSONDecodeError as e:
                print("\nInvalid JSON received:")
                print(profile_str)
                print(f"JSON Error: {str(e)}")
                raise

            for field in ['Experiences', 'Educations', 'Skills', 'Languages', 'Organizations', 'Interests']:
                if field in profile:
                    profile[field] = self.clean_field(profile[field])

            profile['Label'] = 12
            return profile

        except (openai.APIError, json.JSONDecodeError) as e:
            if retry_count < self.max_retries:
                print(f"\nRetry {retry_count + 1}/{self.max_retries} after error: {str(e)}")
                if 'response' in locals():
                    print("Last response received:", response.choices[0].message.content)
                time.sleep(self.retry_delay * (retry_count + 1))
                return self.generate_single_profile(example_profiles, retry_count + 1)
            else:
                raise Exception(f"Failed to generate profile after {self.max_retries} retries: {str(e)}")

    def validate_profile(self, profile: Dict) -> bool:
        try:
            required_fields = ['Full Name', 'Workplace', 'Location', 'Experiences', 'Educations']
            for field in required_fields:
                if field not in profile:
                    print(f"Missing required field: {field}")
                    return False

            for col in self.columns:
                if col not in profile:
                    profile[col] = {} if 'Number' not in col else 0

            return True

        except Exception as e:
            print(f"Validation error: {str(e)}")
            return False

    def save_profiles(self, profiles: List[Dict]):
        output_df = pd.DataFrame(profiles)
        output_df = output_df[self.columns]
        output_df.to_csv(self.output_file, index=False)
        print(f"\nSaved {len(profiles)} profiles to: {self.output_file}")

    
    
    def generate_profiles(self, example_profiles: List[Dict], num_profiles: int) -> List[Dict]:
        # Load existing generated profiles if they exist
        generated_profiles = []
        if os.path.exists(self.output_file):
            print(f"Loading existing profiles from: {self.output_file}")
            df = pd.read_csv(self.output_file)
            generated_profiles = df.to_dict('records')
            print(f"Loaded {len(generated_profiles)} existing profiles")
    
        start_index = len(generated_profiles)
        remaining_profiles = num_profiles - start_index
    
        if remaining_profiles <= 0:
            print("Target number of profiles already generated!")
            return generated_profiles
    
        print(f"Generating {remaining_profiles} more profiles...")
    
        batch_size = 5
        num_batches = (remaining_profiles + batch_size - 1) // batch_size
    
        with tqdm(total=num_profiles, initial=start_index, desc="Generating profiles") as pbar:
            for batch in range(remaining_profiles // batch_size + (1 if remaining_profiles % batch_size else 0)):
                print(f"\nStarting batch {batch + 1}/{num_batches}")
                
                # Select new example profiles for each batch
                example_batch = self.select_training_examples(example_profiles)
                print(f"Selected 5 new random example profiles for this batch")
            
                batch_profiles = []
                failed_generations = 0
                max_failures_per_batch = batch_size * 2
    
                while len(batch_profiles) < batch_size and failed_generations < max_failures_per_batch:
                    try:
                        # Use the same example_batch within this batch
                        profile = self.generate_single_profile(example_batch)
    
                        if self.validate_profile(profile):
                            batch_profiles.append(profile)
                            pbar.update(1)
                        else:
                            failed_generations += 1
                            print(f"\nProfile validation failed. Attempts remaining: {max_failures_per_batch - failed_generations}")
    
                    except Exception as e:
                        failed_generations += 1
                        print(f"\nError generating profile: {str(e)}")
                        print(f"Attempts remaining: {max_failures_per_batch - failed_generations}")
    
                    time.sleep(1)
    
                generated_profiles.extend(batch_profiles)

                # Save progress after each batch
                if batch_profiles:
                    self.save_profiles(generated_profiles)
    
                print(f"Completed batch {batch + 1}, total profiles generated: {len(generated_profiles)}")
    
                if batch < num_batches - 1:
                    time.sleep(2)
    
        return generated_profiles

def main():
    api_key = "INSERT_API_KEY"
    input_file = "cleaned_profiles.csv"
    test_mode = False

    generator = LinkedInProfileGenerator(api_key=api_key)

    print("Loading existing profiles...")
    df = pd.read_csv(input_file)
    existing_profiles = df.to_dict('records')

    print(f"Total profiles loaded: {len(existing_profiles)}")
    print("Label distribution:")
    print(df['Label'].value_counts())

    num_profiles = 10 if test_mode else 1800
    print(f"\nTarget number of profiles: {num_profiles} {'(TEST MODE)' if test_mode else ''}")

    new_profiles = generator.generate_profiles(
        example_profiles=existing_profiles,
        num_profiles=num_profiles
    )

    print(f"\nGeneration complete!")
    print(f"Successfully generated: {len(new_profiles)} profiles")

if __name__ == "__main__":
    main()

Loading existing profiles...
Total profiles loaded: 3600
Label distribution:
Label
0     1800
1      600
10     600
11     600
Name: count, dtype: int64

Target number of profiles: 1800 
Loading existing profiles from: generated_profiles_NEW.csv
Loaded 240 existing profiles
Generating 1560 more profiles...


Generating profiles:  13%|██████▎                                        | 240/1800 [00:00<?, ?it/s]


Starting batch 1/312
Selected 5 new random example profiles for this batch


Generating profiles:  14%|████▉                               | 245/1800 [03:51<19:57:00, 46.19s/it]


Saved 245 profiles to: generated_profiles_NEW.csv
Completed batch 1, total profiles generated: 245

Starting batch 2/312
Selected 5 new random example profiles for this batch


...


Saved 675 profiles to: generated_profiles_NEW.csv
Completed batch 87, total profiles generated: 675

Starting batch 88/312
Selected 5 new random example profiles for this batch


Generating profiles:  38%|████████████▊                     | 680/1800 [6:41:56<18:17:42, 58.81s/it]


Saved 680 profiles to: generated_profiles_NEW.csv
Completed batch 88, total profiles generated: 680

Starting batch 89/312
Selected 5 new random example profiles for this batch


Generating profiles:  38%|████████████▊                     | 680/1800 [6:42:01<17:03:20, 54.82s/it]


KeyboardInterrupt: 